# Create Agents for LLM

# Method 1

In [5]:
import datetime

In [1]:
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
from codes.llm_gateway import openAI_wo_api

In [3]:
llm = openAI_wo_api()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [4]:
@tool
def add(a: int, b: int) -> int:
    """Adds two numbers together""" # this docstring gets used as the description
    return a + b # the actions our tool performs

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers together""" # this docstring gets used as the description
    return a * b # the actions our tool performs

@tool
def power(a: int, b: int) -> int:
    """Returns 'a' raised to the power 'b'""" # this docstring gets used as the description
    return a**b # the actions our tool performs

In [27]:
def text_to_date(text:str):
    '''
    Parses the date given in string to a date format
    '''
    date_in_ = datetime.date.fromisoformat(text)
    return date_in_
    
@tool
def years_between(dob:str, date_on:str=''):
    """
    calculates the difference between the two dates in years
    """
    dob = text_to_date(dob)
    date_on = datetime.date.today() if date_on=='' else text_to_date(date_on)
    age = (date_on - dob).days
    age = round(age / 365, 1)
    return age

In [28]:
toolkit = [add, multiply, power, years_between]

In [29]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
          You are a mathematical assistant. Use your tools to answer questions.
          If you do not have a tool to answer the question, say so. 
        
          Return only the answers. e.g
          Human: What is 1 + 1?
          AI: 2
          """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [30]:
agent = create_openai_tools_agent(llm, toolkit, prompt)

In [31]:
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

In [11]:
result = agent_executor.invoke({"input": "what is 1 + 1"})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 1, 'b': 1}`


22

> Finished chain.


In [12]:
print(result['output'])

2


In [34]:
result = agent_executor.invoke({"input": "I am born on 1986-01-19. What is my age on 2021-12-15?"})
# result = agent_executor.invoke({"input": "I am born on 1986-01-19. What is my age today?"})



> Entering new AgentExecutor chain...

Invoking: `years_between` with `{'dob': '1986-01-19', 'date_on': '2021-12-15'}`


35.935

> Finished chain.


In [35]:
print(result['output'])

35


## Prompt Hub

In [22]:
from langchain import hub

### Example 1

In [22]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Example 2

In [24]:
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [29]:
type(prompt)
print(prompt.messages[0].prompt.template)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


### Example 3

In [36]:
prompt = hub.pull("smithing-gold/question-decomposition")

In [39]:
print(prompt.template)

I’m going to ask you a question. I want you to decompose it into a series of subquestions. Each subquestion should be self-contained with all the information necessary to solve it.

Make sure not to decompose more than necessary or have any trivial subquestions - you’ll be evaluated on the simplicity, conciseness, and correctness of your decompositions as well as your final answer. You should wrap each subquestion in <sub q></sub q> tags. After each subquestion, you should answer the subquestion and put your subanswer in <sub a></sub a> tags.

 Once you have all the information you need to answer the question, output <FIN></FIN> tags.

example:
Question: What is Bitcoin?
<sub q>What is the purpose of Bitcoin?</sub q>
<sub a>Bitcoin serves as a decentralized digital currency.</sub a>
<sub q>What does decentralized mean?</sub q>
<sub a>Decentralized means it operates without a central authority or single administrator.</sub a>
<FIN>Bitcoin is a decentralized digital currency that operate

### Exampe 4

In [40]:
prompt = hub.pull("rlm/text-to-sql")

In [43]:
print(prompt.messages[0].prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Some examples of SQL queries that corrsespond to questions are:

{few_shot_examples}

Question: {input}


## Other Scenario

In [37]:
import pandas as pd

In [38]:
df_test = pd.DataFrame({
    'id': [1001, 2001, 3001, 4001, 5001],
    'name': ['Adam Zohe', 'Bevan Young', 'Clarie Xong', 'Daniel Wells', 'Ema Villing'],
    'position': ['pos2', 'pos3', 'pos4', 'pos3', 'pos3'],
    'sales': [100, 140, 180, 130, 120],
})

df_test

,id,name,position,sales
0,1001,Adam Zohe,pos2,100
1,2001,Bevan Young,pos3,140
2,3001,Clarie Xong,pos4,180
3,4001,Daniel Wells,pos3,130
4,5001,Ema Villing,pos3,120


In [39]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_experimental.text_splitter import SemanticChunker

In [40]:
agent = create_pandas_dataframe_agent(llm, df_test, verbose=True)

In [42]:
query = "how big is the dataset?"
result = agent.invoke(query)



> Entering new AgentExecutor chain...
Thought: To find out the size of the dataset, I need to check the number of rows and columns in the dataframe `df`.

Action: python_repl_ast
Action Input: df.shape
(5, 4)I now know the final answer.

Final Answer: The dataset contains 5 rows and 4 columns.

> Finished chain.


In [43]:
print(result['output'])

The dataset contains 5 rows and 4 columns.


In [59]:
query = "Is there a difference in the sales of people from different positions?"
result = agent.invoke(query)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to group the data by the 'position' column and then calculate the sum or average sales for each position to see if there's a difference.

Action: I'll use the groupby() function on the 'position' column and then calculate the mean sales using the mean() function.

Action Input: df.groupby('position')['sales'].mean()I'll use the groupby() function on the 'position' column and then calculate the mean sales using the mean() function. is not a valid tool, try one of [python_repl_ast].I need to use the provided Python shell to execute the command instead of stating what I intend to do. 

Action: I will use the python_repl_ast to execute the command that will group the dataframe by the 'position' column and calculate the mean sales for each position.

Action Input: df.groupby('position')['sales'].mean()I will use the python_repl_ast to execute the command that will group the dataframe by the 'position' column an

In [60]:
print(result['output'])

Yes, there is a difference in the sales of people from different positions. The average sales for 'pos2' is 100, for 'pos3' is 130, and for 'pos4' is 180.


In [58]:
query = "What is the average sales from different positions?"
result = agent.invoke(query)



> Entering new AgentExecutor chain...
Thought: To find the average sales per position, I can use the `groupby` function on the 'position' column and then calculate the mean of the 'sales' for each group.

Action: python_repl_ast
Action Input: import pandas as pd
# Assuming df is the DataFrame in question
df = pd.DataFrame({
    'id': [1001, 2001, 3001, 4001, 5001],
    'name': ['Adam Zohe', 'Bevan Young', 'Clarie Xong', 'Daniel Wells', 'Ema Villing'],
    'position': ['pos2', 'pos3', 'pos4', 'pos3', 'pos3'],
    'sales': [100, 140, 180, 130, 120]
})
df.groupby('position')['sales'].mean()position
pos2    100.0
pos3    130.0
pos4    180.0
Name: sales, dtype: float64I now know the final answer
Final Answer: The average sales from different positions are as follows:
- Position pos2: 100.0
- Position pos3: 130.0
- Position pos4: 180.0

> Finished chain.


In [53]:
print(result['output'])

The average sales from different positions are as follows: pos2 has an average sale of 100.0, pos3 has an average sale of 130.0, and pos4 has an average sale of 180.0.


In [1]:
text = """Thank you for submitting your request. Hello $contacts.alt_name.first  Thank you for your interest accessing Walmart Luminate Channel Performance>Report Builder. To which Strategic Business Unit (SBU) or Units do you need access? The available SBUs are: •	Apparel •	Consumables •	Food •	General Merchandise •	Health and Wellness •	Walmart Services  It is possible for an associate to get access to ALL SBUs but, to maintain legal compliance, the associate needs to be added to the SEC Insider Trading Blackout List. Adding someone to the Blackout List requires Manager Approval to be documented via email. In case you do need access to ALL SBU's, I am adding <Manager Name in BOLD> to "Reply all" to this email and provide an "I approve" or "I do not approve" response.  We will move forward with creating your account once we have received everyone's response and know which SBU or SBU's to add.  If you wish to access the Shopper Behavior application within Walmart Luminate, please navigate to wmlink/ulearn (https://moodle-azure.prod.walmart.com/moodle/my/) , search for Shopper Behavior, and sign up for a Shopper Behavior: Quick Start course. Access will be granted once the course has been completed.  Customer Perception is used to survey a pool of verified Walmart Customers regarding products and purchase history. Access is restricted to signed Suppliers and select WMT Luminate Support personnel. Internal associate led research projects are managed through the Conversation Ally team. Please review the Best Practices and Qualitative and Quantitative Briefs links below to learn more about submitting a survey request to the Conversation Ally team:  https://one.walmart.com/content/conversation_ally/best-practices.html https://one.walmart.com/content/conversation_ally/quant_qual_briefs.html   Walmart Associates are permissioned to Walmart Luminate by signing up for Shopper Behavior Quick Start in Ulearn. : https://moodle-azure.prod.walmart.com/moodle/mod/facetoface/view.php?id=17168  Associates can also check out wmlink/Walmart-Luminate-for-Merchants to see the Merchandising Resource Center as well.  Once you receive access, you’ll log into the platform at WalmartLuminate.com with Single Sign On (SSO). You will need to be at the office or on VPN.  Thank you, Walmart Luminate Support """

In [2]:
print(text)

Thank you for submitting your request. Hello $contacts.alt_name.first  Thank you for your interest accessing Walmart Luminate Channel Performance>Report Builder. To which Strategic Business Unit (SBU) or Units do you need access? The available SBUs are: •	Apparel •	Consumables •	Food •	General Merchandise •	Health and Wellness •	Walmart Services  It is possible for an associate to get access to ALL SBUs but, to maintain legal compliance, the associate needs to be added to the SEC Insider Trading Blackout List. Adding someone to the Blackout List requires Manager Approval to be documented via email. In case you do need access to ALL SBU's, I am adding <Manager Name in BOLD> to "Reply all" to this email and provide an "I approve" or "I do not approve" response.  We will move forward with creating your account once we have received everyone's response and know which SBU or SBU's to add.  If you wish to access the Shopper Behavior application within Walmart Luminate, please navigate to wml